In [1]:
from langchain_community.vectorstores import FAISS
import os 
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings 
from langgraph.graph import START, END, StateGraph
from langchain.schema import Document
from typing import TypedDict, List
from dotenv import load_dotenv
from langchain_community.embeddings import HuggingFaceEmbeddings

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(model="gpt-4.1", temperature=0)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

class AgentState(TypedDict):
    question: str
    documents: List[Document]
    answer: str
    needs_retrieval: bool     # ✅ fixed key name

sample_texts = [
    "Effective financial management helps organizations allocate resources efficiently and achieve long-term goals.",
    "Budgeting is a crucial part of financial management that ensures spending aligns with income.",
    "Cash flow management helps businesses maintain liquidity and avoid short-term financial crises.",
    "Good financial decisions are based on analyzing risks, returns, and market trends.",
    "Personal financial management involves saving, investing, and controlling expenses wisely.",
]

documents = [Document(page_content=text) for text in sample_texts]

# create vector store
vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever(k=3)

def decide_retrieval(state: AgentState) -> AgentState:      # ✅ fixed function name spelling
    """decide when to retrieve documents based on question"""
    question = state['question']
    retrieval_keywords = ["what", "how", "explain", "describe", "tell me"]
    needs_retrieval = any(keyword in question.lower() for keyword in retrieval_keywords)

    return {**state, "needs_retrieval": needs_retrieval}    # ✅ fixed key name

def retrieve_documents(state: AgentState) -> AgentState:
    """retrieve relevant document based on question"""
    question = state["question"]
    documents = retriever.invoke(question)

    return {**state, "documents": documents}                 # ✅ fixed key name

def generate_answer(state: AgentState) -> AgentState:
    """retrieve the answer based on documents or direct answer"""
    question = state["question"]
    documents = state.get("documents", [])                   # ✅ fixed from retriever.get

    if documents:
        context = "\n\n".join([doc.page_content for doc in documents])
        prompt = f"""based on the following context answer the question
    Context:
    {context}

    Question: {question}

    Answer:"""
    else:
        prompt = f"answer the following question: {question}"

    response = llm.invoke(prompt)
    answer = response.content

    return {**state, "answer": answer}

def should_retrieve(state: AgentState) -> str:
    """determine the next step based on retrieval decision"""
    if state["needs_retrieval"]:                             # ✅ consistent key
        return "retrieve"
    else:
        return "generate"

workflow = StateGraph(AgentState)

workflow.add_node("decide", decide_retrieval)
workflow.add_node("retrieve", retrieve_documents)
workflow.add_node("generate", generate_answer)

workflow.set_entry_point("decide")

workflow.add_conditional_edges(
    "decide",
    should_retrieve,
    {
        "retrieve": "retrieve",
        "generate": "generate"
    }
)

workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", END)

app = workflow.compile()
app

def ask_question(question: str):
    """
    help function to ask a question and get an answer
    """
    initial_state = {
        "question": question,
        "documents": [],
        "answer": "",
        "needs_retrieval": False                              # ✅ consistent key
    }

    result = app.invoke(initial_state)
    return result

question1 = "tell me about financial management"
result1 = ask_question(question1)
result1


d:\langchain2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\ASUS\AppData\Local\Temp\ipykernel_11064\1770743754.py:15: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}